# Serving a Keras Model on AI Platform Prediction with request-response logging to BigQuery

This tutorial shows how to train a TensorFlow classification model using the Keras API, and then deploy the model to AI Platform Prediction for online prediction. The tutorial also shows how to enable [AI Platform Prediction request-response logging](https://cloud.google.com/ai-platform/prediction/docs/online-predict#requesting_logs_for_online_prediction_requests) to BigQuery.

The tutorial covers the following tasks:

1. Prepare the data and generate metadata. 
2. Train and evaluate a TensorFlow classification model using the Keras API.
3. Export the trained model as a SavedModel for serving.
4. Deploy the trained model to AI Platform Prediction.
5. Enable request-response logging to send logs to BigQuery.
6. Query logs from BigQuery.


**Note**: This example uses **TensorFlow 2.x**

## Setup

### Install packages and dependencies

In [ ]:
!pip install -q -U tensorflow==2.1
!pip install -U -q google-api-python-client
!pip install -U -q pandas

In [ ]:
# Automatically restart kernel after installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)  

### Configure Google Cloud environment settings



In [ ]:
PROJECT_ID = '[your-google-project-id]'
BUCKET = '[your-bucket-name]'
REGION = '[your-region-id]'
!gcloud config set project $PROJECT_ID

### Authenticate your Google Cloud account

This step is required if you run the notebook in Colab.

In [ ]:
try:
  from google.colab import auth
  auth.authenticate_user()
  print("Colab user is authenticated.")
except: pass

### Import libraries

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import tensorflow as tf
import pandas as pd
from google.cloud import bigquery

print("TF version: {}".format(tf.__version__))

### Define constants

You can change the default values for the following constants:


In [ ]:
LOCAL_WORKSPACE = './workspace'
LOCAL_DATA_DIR = os.path.join(LOCAL_WORKSPACE, 'data')
BQ_DATASET_NAME = 'prediction_logs'
BQ_TABLE_NAME = 'covertype_classifier_logs' 
MODEL_NAME = 'covertype_classifier'
VERSION_NAME = 'v1' 
TRAINING_DIR = os.path.join(LOCAL_WORKSPACE, 'training')
MODEL_DIR = os.path.join(TRAINING_DIR, 'exported_model')

### Create a local workspace

In [ ]:
if tf.io.gfile.exists(LOCAL_WORKSPACE):
  print("Removing previous workspace artifacts...")
  tf.io.gfile.rmtree(LOCAL_WORKSPACE)

print("Creating a new workspace...")
tf.io.gfile.makedirs(LOCAL_WORKSPACE)
tf.io.gfile.makedirs(LOCAL_DATA_DIR)
print("Workspace created.")

## 1. Preparing the dataset and defining the metadata

The data in this tutorial is based on the [covertype](https://archive.ics.uci.edu/ml/datasets/covertype) dataset from UCI Machine Learning Repository. The notebook uses a version of the dataset that has been preprocessed, split, and uploaded to a public Cloud Storage bucket at the following location: 

`gs://workshop-datasets/covertype`

For more information, see [Cover Type Dataset](https://github.com/GoogleCloudPlatform/mlops-on-gcp/tree/master/datasets/covertype)

The task in this tutorial is to predict forest cover type from cartographic variables only. The aim is to build and deploy a **minimal model** to showcase the AI Platform Prediction request-response logging capabilities. Such logs let you perform further analysis for detecting data skews.

### 1.1. Download the data

In [ ]:
LOCAL_TRAIN_DATA = os.path.join(LOCAL_DATA_DIR, 'train.csv') 
LOCAL_EVAL_DATA = os.path.join(LOCAL_DATA_DIR, 'eval.csv') 

In [ ]:
!gsutil cp gs://workshop-datasets/covertype/data_validation/training/dataset.csv {LOCAL_TRAIN_DATA}
!gsutil cp gs://workshop-datasets/covertype/data_validation/evaluation/dataset.csv {LOCAL_EVAL_DATA}
!wc -l {LOCAL_TRAIN_DATA}

View a sample of the downloaded data:

In [ ]:
pd.read_csv(LOCAL_TRAIN_DATA).head().T

### 1.2 Define the metadata
The following code shows the metadata of the dataset, which is used to create the data input function, the feature columns, and the serving function.

In [ ]:
HEADER = ['Elevation', 'Aspect', 'Slope','Horizontal_Distance_To_Hydrology',
          'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
          'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
          'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area', 'Soil_Type',
          'Cover_Type']

TARGET_FEATURE_NAME = 'Cover_Type'

TARGET_FEATURE_LABELS = ['0', '1', '2', '3', '4', '5', '6']

NUMERIC_FEATURE_NAMES = ['Aspect', 'Elevation', 'Hillshade_3pm', 
                         'Hillshade_9am', 'Hillshade_Noon', 
                         'Horizontal_Distance_To_Fire_Points',
                         'Horizontal_Distance_To_Hydrology',
                         'Horizontal_Distance_To_Roadways','Slope',
                         'Vertical_Distance_To_Hydrology']

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    'Soil_Type': ['2702', '2703', '2704', '2705', '2706', '2717', '3501', '3502', 
                  '4201', '4703', '4704', '4744', '4758', '5101', '6101', '6102', 
                  '6731', '7101', '7102', '7103', '7201', '7202', '7700', '7701', 
                  '7702', '7709', '7710', '7745', '7746', '7755', '7756', '7757', 
                  '7790', '8703', '8707', '8708', '8771', '8772', '8776'], 
    'Wilderness_Area': ['Cache', 'Commanche', 'Neota', 'Rawah']
}

FEATURE_NAMES = list(CATEGORICAL_FEATURES_WITH_VOCABULARY.keys()) + NUMERIC_FEATURE_NAMES

HEADER_DEFAULTS = [[0] if feature_name in NUMERIC_FEATURE_NAMES + [TARGET_FEATURE_NAME] else ['NA'] 
                   for feature_name in HEADER]

NUM_CLASSES = len(TARGET_FEATURE_LABELS)

## 2. Training and evaluating the model

### 2.1. Implement the data input pipeline

In [ ]:
RANDOM_SEED = 19830610
import multiprocessing

def create_dataset(file_pattern, 
                  batch_size=128, num_epochs=1, shuffle=False):
  
    dataset = tf.data.experimental.make_csv_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        column_names=HEADER,
        column_defaults=HEADER_DEFAULTS,
        label_name=TARGET_FEATURE_NAME,
        field_delim=',',
        header=True,
        num_epochs=num_epochs,
        shuffle=shuffle,
        shuffle_buffer_size=(5 * batch_size),
        shuffle_seed=RANDOM_SEED,
        num_parallel_reads=multiprocessing.cpu_count(),
        sloppy=True,
    )
    return dataset.cache()

The following code performs a test by reading some batches of data using the data input function:

In [ ]:
index = 1
for batch in create_dataset(LOCAL_TRAIN_DATA, batch_size=5, shuffle=False).take(2):
  print("Batch: {}".format(index))
  print("========================")
  record, target = batch
  print("Input features:")
  for key in record:
    print(" - {}:{}".format(key, record[key].numpy()))
  print("Target: {}".format(target))
  index += 1
  print()

### 2.2. Create feature columns

In [ ]:
import math

def create_feature_columns():
  feature_columns = []
  
  for feature_name in FEATURE_NAMES:
    # Categorical features
    if feature_name in CATEGORICAL_FEATURES_WITH_VOCABULARY:
      
      vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
      vocab_size = len(vocabulary)
      
      # Create embedding column for categorical feature column with vocabulary
      embedding_feature_column = tf.feature_column.embedding_column(
          categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(
              key=feature_name,
              vocabulary_list=vocabulary), dimension=int(math.sqrt(vocab_size) + 1))
            
      feature_columns.append(embedding_feature_column)

    # Numeric features
    else:
      numeric_column = tf.feature_column.numeric_column(feature_name)
      feature_columns.append(numeric_column)

  return feature_columns


The following code tests the feature columns to be created:

In [ ]:
feature_columns = create_feature_columns()

for column in feature_columns:
  print(column)

### 2.3. Create and compile the model



In [ ]:
def create_model(params):

  feature_columns = create_feature_columns()
  
  layers = []
  layers.append(tf.keras.layers.DenseFeatures(feature_columns))
  for units in params.hidden_units:
    layers.append(tf.keras.layers.Dense(units=units, activation='relu'))
    layers.append(tf.keras.layers.BatchNormalization())
    layers.append(tf.keras.layers.Dropout(rate=params.dropout))
  
  layers.append(tf.keras.layers.Dense(units=NUM_CLASSES, activation='softmax'))
  
  model = tf.keras.Sequential(layers=layers, name='classifier')
    
  adam_optimzer = tf.keras.optimizers.Adam(learning_rate=params.learning_rate)

  model.compile(
        optimizer=adam_optimzer, 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()], 
        loss_weights=None,
        sample_weight_mode=None, 
        weighted_metrics=None, 
    )

  return model  

### 2.4. Train and evaluate the experiment

#### Define the experiment

In [ ]:
def run_experiment(model, params):

  # TensorBoard callback
  LOG_DIR = os.path.join(TRAINING_DIR, 'logs')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR)

  # Early stopping callback
  earlystopping_callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_sparse_categorical_accuracy', 
      patience=3, 
      restore_best_weights=True
  )

  callbacks = [
        tensorboard_callback,
        earlystopping_callback]

  # Train dataset
  train_dataset = create_dataset(
      LOCAL_TRAIN_DATA,
      batch_size=params.batch_size,
      shuffle=True)
    
  # Eval dataset
  eval_dataset = create_dataset(
      LOCAL_EVAL_DATA,
      batch_size=params.batch_size)
    
  # Prep training directory
  if tf.io.gfile.exists(TRAINING_DIR):
    print("Removing previous training artifacts...")
    tf.io.gfile.rmtree(TRAINING_DIR)

  print("Creating training directory...")
  tf.io.gfile.mkdir(TRAINING_DIR)

  print("Experiment started...")
  print(".......................................")
  
  # Run train and evaluate
  history = model.fit(
    x=train_dataset, 
    epochs=params.epochs, 
    callbacks=callbacks,
    validation_data=eval_dataset,
  )

  print(".......................................")
  print("Experiment finished.")
  print("")

  return history


#### Define hyperparameters

In [ ]:
class Parameters():
    pass

TRAIN_DATA_SIZE = 431010

params = Parameters()
params.learning_rate = 0.01
params.hidden_units = [128, 128]
params.dropout = 0.15
params.batch_size =  265
params.steps_per_epoch = int(math.ceil(TRAIN_DATA_SIZE / params.batch_size))
params.epochs = 10

#### Run the experiment

In [ ]:
model = create_model(params)
example_batch, _ = list(
    create_dataset(LOCAL_TRAIN_DATA, batch_size=2, shuffle=True).take(1))[0]
model(example_batch)
model.summary()

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

history = run_experiment(model, params)

#### Visualize training history

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_size_inches(w=(10, 5))

# Plot training & validation accuracy values
ax1.plot(history.history['sparse_categorical_accuracy'])
ax1.plot(history.history['val_sparse_categorical_accuracy'])
ax1.set_title('Model Accuracy')
ax1.set(xlabel='Iteration', ylabel='accuracy')
ax1.legend(['Train', 'Eval'], loc='upper left')

# Plot training & validation loss values
ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_title('Model Loss')
ax2.set(xlabel='Iteration', ylabel='loss')
ax2.legend(['Train', 'Eval'], loc='upper left')

## 3. Export the model for serving

In [ ]:
LABEL_KEY = 'predicted_label'
SCORE_KEY = 'confidence'
PROBABILITIES_KEY = 'probabilities'
SIGNATURE_NAME = 'serving_default'

### 3.1. Implement serving input receiver functions

#### Create the serving function

The notebook creates a serving input function that expects a features dictionary and returns the following:
- Predicted class label
- Prediction confidence
- Prediction probabilities of all the classes

In [ ]:
def make_features_serving_fn(model):

  @tf.function
  def serve_features_fn(features):
    probabilities = model(features)
    labels = tf.constant(TARGET_FEATURE_LABELS, dtype=tf.string)
    predicted_class_indices = tf.argmax(probabilities, axis=1)
    predicted_class_label = tf.gather(
        params=labels, indices=predicted_class_indices)
    prediction_confidence = tf.reduce_max(probabilities, axis=1)
    
    return {
        LABEL_KEY: predicted_class_label,
        SCORE_KEY:prediction_confidence,
        PROBABILITIES_KEY: probabilities}

  return serve_features_fn

#### Create the feature spec dictionary

The code creates the `feature_spec` dictionary for the input features with respect to the dataset metadata:

In [ ]:
feature_spec = {}
for feature_name in FEATURE_NAMES:
    if feature_name in CATEGORICAL_FEATURES_WITH_VOCABULARY:
        feature_spec[feature_name] = tf.io.FixedLenFeature(
            shape=[None], dtype=tf.string)
    else:
        feature_spec[feature_name] = tf.io.FixedLenFeature(
            shape=[None], dtype=tf.float32)

for key, value in feature_spec.items():
  print("{}: {}".format(key, value))

### 3.2. Export the model

In [ ]:
features_input_signature = {
    feature: tf.TensorSpec(shape=spec.shape, dtype=spec.dtype, name=feature)
    for feature, spec in feature_spec.items()}

signatures = {        
    SIGNATURE_NAME: make_features_serving_fn(model).get_concrete_function(
        features_input_signature)}

model.save(MODEL_DIR, save_format='tf', signatures=signatures)
print("Model is exported to: {}.".format(MODEL_DIR))

Verify the signature (inputs and outputs) of the exported model using the`saved_model_cli` function:

In [ ]:
!saved_model_cli show --dir {MODEL_DIR} --tag_set serve --signature_def {SIGNATURE_NAME}

### 3.3. Test the exported model locally

Create a sample instance for prediction:

In [ ]:
instances = [
      { 
        'Soil_Type': ['7202'],
        'Wilderness_Area': ['Commanche'],
        'Aspect': [61],
        'Elevation': [3091],
        'Hillshade_3pm': [129],
        'Hillshade_9am': [227],
        'Hillshade_Noon': [223],
        'Horizontal_Distance_To_Fire_Points': [2868],
        'Horizontal_Distance_To_Hydrology': [134],
        'Horizontal_Distance_To_Roadways': [0], 
        'Slope': [8], 
        'Vertical_Distance_To_Hydrology': [10],
    }
]

Prepare the sample instance in the format that's expected by the model signature:

In [ ]:
import numpy as np

def create_tf_features(instance):
 
  new_instance = {}
  for key, value in instance.items():
    if key in CATEGORICAL_FEATURES_WITH_VOCABULARY:
      new_instance[key] = tf.constant(value, dtype=tf.string)
    else:
      new_instance[key] = tf.constant(value, dtype=tf.float32)
  
  return new_instance

Load the SavedModel for prediction, and then create a function that generates the prediction probabilities from the model to return the class label that has the highest probability:

In [ ]:
features_predictor = tf.saved_model.load(MODEL_DIR).signatures[SIGNATURE_NAME]

def local_predict(instance):
  features = create_tf_features(instance)
  outputs = features_predictor(**features)
  return outputs 

Perform a prediction using the local SavedModel:

In [ ]:
outputs = local_predict(instances[0])
predictions = list(
    zip(outputs[LABEL_KEY].numpy().tolist(), 
        outputs[SCORE_KEY].numpy().tolist()))

for prediction in predictions:
  print("Predicted label: {} - Prediction confidence: {}".format(
        prediction[0], round(prediction[1], 3)))

### 3.4  Upload the exported model to Cloud Storage

In [ ]:
!gsutil rm -r gs://{BUCKET}/models/{MODEL_NAME}
!gsutil cp -r {MODEL_DIR} gs://{BUCKET}/models/{MODEL_NAME}

## 4. Deploy the model to AI Platform Prediction


### 4.1. Create the model in AI Platform Prediction

In [ ]:
!gcloud ai-platform models create {MODEL_NAME} \
  --project {PROJECT_ID} \
  --regions {REGION}

# List the models
!gcloud ai-platform models list --project {PROJECT_ID}

### 4.2. Create a model version

In [ ]:
!gcloud ai-platform versions create {VERSION_NAME} \
  --model={MODEL_NAME} \
  --origin=gs://{BUCKET}/models/{MODEL_NAME} \
  --runtime-version=2.1 \
  --framework=TENSORFLOW \
  --python-version=3.7 \
  --project={PROJECT_ID}

# List the model versions
!gcloud ai-platform versions list --model={MODEL_NAME} --project={PROJECT_ID}

### 4.3. Test the deployed model

Create a function to call the AI Platform Prediction model version:

In [ ]:
import googleapiclient.discovery

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, VERSION_NAME)
print("Service name: {}".format(name))

def caip_predict(instances):
  
  request_body={
      'signature_name': SIGNATURE_NAME,
      'instances': instances}

  response = service.projects().predict(
      name=name,
      body=request_body

  ).execute()

  if 'error' in response:
    raise RuntimeError(response['error'])

  outputs = response['predictions']
  return outputs

Perform a prediction using AI Platform Prediction:

In [ ]:
outputs = caip_predict(instances)
for output in outputs:
  print("Predicted label: {} - Prediction confidence: {}".format(
        output[LABEL_KEY], round(output[SCORE_KEY], 3)))

## 5. Preparing logging for a BigQuery dataset

### 5.1. Create the BigQuery dataset

In [ ]:
client = bigquery.Client(PROJECT_ID)
dataset_names = [dataset.dataset_id for dataset in client.list_datasets(PROJECT_ID)]

dataset = bigquery.Dataset("{}.{}".format(PROJECT_ID, BQ_DATASET_NAME))
dataset.location = "US"

if BQ_DATASET_NAME not in dataset_names:
  dataset = client.create_dataset(dataset)
  print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

print("BigQuery dataset is ready.")

### 5.2. Create the BigQuery table to store the logs


#### Define the table schema

In [ ]:
import json

table_schema_json = [
  {
    "name": "model", 
    "type": "STRING", 
    "mode": "REQUIRED"
   },
   {
     "name":"model_version", 
     "type": "STRING", 
     "mode":"REQUIRED"
  },
  {
    "name":"time", 
    "type": "TIMESTAMP", 
    "mode": "REQUIRED"
  },
  {
    "name":"raw_data", 
    "type": "STRING", 
    "mode": "REQUIRED"
  },
  {
    "name":"raw_prediction", 
    "type": "STRING", 
    "mode": "NULLABLE"
  },
  {
    "name":"groundtruth", 
    "type": "STRING", 
    "mode": "NULLABLE"
  },
]

json.dump(
    table_schema_json, open('table_schema.json', 'w'))

#### Create a table that's partitioned on ingestion time

In [ ]:
table = bigquery.Table(
    "{}.{}.{}".format(PROJECT_ID, BQ_DATASET_NAME, BQ_TABLE_NAME))

table_names = [table.table_id for table in client.list_tables(dataset)]

if BQ_TABLE_NAME in table_names:
  print("Deleting BQ table: {} ...".format(BQ_TABLE_NAME))
  client.delete_table(table)

In [ ]:
TIME_PARTITION_EXPERIATION = int(60 * 60 * 24 * 7)

!bq mk --table \
  --project_id={PROJECT_ID} \
  --time_partitioning_field=time \
  --time_partitioning_type=DAY \
  --time_partitioning_expiration={TIME_PARTITION_EXPERIATION} \
  {PROJECT_ID}:{BQ_DATASET_NAME}.{BQ_TABLE_NAME} \
  'table_schema.json'

### 5.3. Configure the AI Platform Prediction model version to enable request-response logging to BigQuery

In order to enable the request-response logging to an existing AI Platform Prediction model version, you need to call the `patch` API and populate the [`requestLoggingConfig`](https://cloud.google.com/ai-platform/prediction/docs/online-predict#requesting_logs_for_online_prediction_requests) field.

In [ ]:
sampling_percentage = 1.0
bq_full_table_name = '{}.{}.{}'.format(PROJECT_ID, BQ_DATASET_NAME, BQ_TABLE_NAME)

In [ ]:
logging_config = {
    "requestLoggingConfig":{
        "samplingPercentage": sampling_percentage,
        "bigqueryTableName": bq_full_table_name
        }
    }

service.projects().models().versions().patch(
    name=name,
    body=logging_config,
    updateMask="requestLoggingConfig"
    ).execute()

### 5.4. Test request-response logging

Send sample prediction requests to the model version on AI Platform Prediction:

In [ ]:
import time

for i in range(5):
  caip_predict(instances)
  print('.', end='')
  time.sleep(1)

Query the logged request-response entries in BigQuery:

In [ ]:
query = '''
  SELECT * FROM 
  `{}.{}` 
  WHERE model_version = '{}'
  ORDER BY time desc
  LIMIT {}
'''.format(BQ_DATASET_NAME, BQ_TABLE_NAME, VERSION_NAME, 3)

pd.io.gbq.read_gbq(
    query, project_id=PROJECT_ID).T
